In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


First run `chargpt.py` then this notebook

In [36]:
from mingpt.utils import set_seed
import torch
from mingpt.model import GPT
import projects.chargpt.chargpt as chargpt

set_seed(3407)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [37]:
config = chargpt.get_config()
text = open("input.txt", "r").read()
train_dataset = chargpt.CharDataset(config.data, text)
config.model.vocab_size = train_dataset.get_vocab_size()
config.model.block_size = train_dataset.get_block_size()
model = GPT(config.model).to(device)
model.eval()
weights = torch.load("out/chargpt/model.pt")
model.load_state_dict(weights)

data has 77358 characters, 61 unique.
number of parameters: 2.71M


<All keys matched successfully>

In [48]:
text[:15]

'First Citizen:\n'

In [49]:
context = text[:15]
encode = [train_dataset.stoi[s] for s in context]
x = torch.tensor(encode, dtype=torch.long)[None, ...].to(device)

In [50]:
# We will make B generations for the same prompt
B = 5
x = x.repeat(B, 1)

In [51]:
def decode(tokens: torch.Tensor) -> str:
    return ''.join([train_dataset.itos[int(i)] for i in tokens])

In [56]:
y, log_probs = model.generate(x, 500, temperature=1.0, do_sample=True, top_k=None, do_log_prob=True)

In [57]:
print(("\n" + "-" * 64 + "\n").join([decode(i) for i in y]))

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in 
----------------------------------------------------------------
First Citizen:
And topping all others in boasting.

MENENIUS:
This is strange now: do you two know how you are
censured here in the city, I mean of us o' the
right-hand file? do you?

Both:
Why, how are we censured?

MENENIUS:
Because you talk of pride now,--will you not be angry?

Both:
Well, well, sir, well.

MENENIUS:
Why, 'tis no great matter; for a very little thief of
occasion will rob you of a great deal of p

In [55]:
perplexity = torch.exp(log_probs.mean(-1))
perplexity

tensor([0.9774, 0.9767, 0.9645, 0.9715, 0.9774], device='cuda:0')